In [2]:
# Install dependencies.
# !pip install asyncio==3.4.3 asyncpg==0.27.0 cloud-sql-python-connector["asyncpg"]==1.2.3
# !pip install numpy==1.22.4 pandas==1.5.3
# !pip install pgvector==0.1.8
# !pip install langchain==0.0.196 transformers
# !pip install google-cloud-aiplatform==1.26.0

In [3]:
# Automatically restart kernel after installs so that your environment
# can access the new packages.
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [4]:
import os
# Please fill in these values.
project_id = "engaged-plasma-439804-k0"  # @param {type:"string"}
database_password = os.getenv('DATABASE_PASSWORD')  # @param {type:"string"}
region = "us-central1"  # @param {type:"string"}
instance_name = "research-paper-db"  # @param {type:"string"}
database_name = "research-papers"  # @param {type:"string"}
database_user = "superuser"  # @param {type:"string"}


# Quick input validations.
assert project_id, "⚠️ Please provide a Google Cloud project ID"
assert region, "⚠️ Please provide a Google Cloud region"
assert instance_name, "⚠️ Please provide the name of your instance"
assert database_name, "⚠️ Please provide a database name"
assert database_user, "⚠️ Please provide a database user"
assert database_password, "⚠️ Please provide a database password"

In [5]:
# @markdown Verify that you are able to connect to the database. Executing this block should print the current PostgreSQL server version.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def test_connection():
    # get current running event loop to be used with Connector
    print(1)
    loop = asyncio.get_running_loop()
    print(1)
    # initialize Connector object as async context manager
    async with Connector(loop=loop) as connector:
        print(1)
        # create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}"
            # ... additional database driver args
        )

        # query Cloud SQL database
        results = await conn.fetch("SELECT version()")
        print(results[0]["version"])

        # close asyncpg connection
        await conn.close()


# Test connection with `asyncio`
await test_connection()  # type: ignore

1
1
1


c:\Users\User\GaTech\hackathon\i-cite\venv\Lib\site-packages\google\cloud\sql\connector\instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by Debian clang version 12.0.1, 64-bit


In [64]:
# Save the Pandas dataframe in a PostgreSQL table.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector


async def reset_database():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("DROP TABLE IF EXISTS papers CASCADE")
        # Create the `products` table.
        await conn.execute(
            """CREATE TABLE papers(
                                doi TEXT PRIMARY KEY,
                                title TEXT,
                                abstract TEXT,
                                content TEXT,
                                citation_text TEXT,
                                embedding vector(1024))"""
        )

        # Copy the dataframe to the `products` table.
        # tuples = list(df.itertuples(index=False))
        # await conn.copy_records_to_table(
        #     "products", records=tuples, columns=list(df), timeout=10
        # )
        await conn.close()


# Run the SQL commands now.
# await reset_database()  # type: ignore

In [156]:
# Store the generated vector embeddings in a PostgreSQL table.
# This code may run for a few minutes.

import asyncio
import asyncpg
from google.cloud.sql.connector import Connector
import numpy as np
from pgvector.asyncpg import register_vector


async def insert_into_papers(papers_df = None):
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
        await register_vector(conn)


        # res = await conn.execute("DROP TABLE IF EXISTS papers CASCADE")
        # print(res)
        # Create the `products` table.
        # await conn.execute(
        #     """CREATE TABLE papers(
        #                         doi TEXT PRIMARY KEY,
        #                         title TEXT,
        #                         abstract TEXT,
        #                         content TEXT,
        #                         citation_text TEXT,
        #                         embedding vector(1024))"""
        # )
        if type(df) == pd.DataFrame:
            # Store all the generated embeddings back into the database.
            insert_values = [
                (
                    str(row["doi"]),
                    row["title"],
                    row["abstract"],
                    row["content"],
                    row["citation_text"],
                    np.array(row["embedding"]),
                    row['id'],
                    np.array(row['title_embedding']),

                )
                for index, row in papers_df.iterrows()
            ]

            # Use executemany for bulk insert
            await conn.executemany(
                "INSERT INTO papers (doi, title, abstract, content, citation_text, embedding, id, title_embedding) VALUES ($1, $2, $3, $4, $5, $6, $7, $8)",
                insert_values
            )

        await conn.close()

# import pandas as pd
# import numpy as np
# df = pd.read_csv("test.csv")
# df["embedding"] = [np.random.rand(1024) for _ in range(len(df))]
# df
# # Run the SQL commands now.
# await insert_into_papers(df)  # type: ignore

In [147]:
async def update_papers(papers_df = None):
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
        await register_vector(conn)

        if type(df) == pd.DataFrame:
            # Store all the generated embeddings back into the database.
            update_values = [
                (
                    str(row["doi"]),
                    row["title_embedding"],
                    row["id"],
                )
                for index, row in papers_df.iterrows()
            ]

            # Use executemany for bulk insert
            await conn.executemany(
                """
                UPDATE papers 
                SET title_embedding = $2, id = $3 
                WHERE doi = $1
                """,
                update_values
            )

        await conn.close()

In [153]:
async def fetch(embeding, nbr_articles=2, DESC=True):
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        # Find similar products to the query using cosine similarity search
        # over all vector embeddings. This new feature is provided by `pgvector`.
        results = await conn.fetch(
            f"""
                            SELECT title, 1 - (embedding <=> $1) AS similarity, abstract, doi, 1 - (title_embedding <=> $1) AS title_similarity
                            FROM papers
                            ORDER BY similarity {"DESC" if DESC else ""}
                            LIMIT $2
                            """,
            embeding,
            nbr_articles
        )

        if len(results) == 0:
            raise Exception("Did not find any results. Adjust the query parameters.")
        matches = []
        for r in results:
            # Collect the description for all the matched similar toy products.
            matches.append(
                {
                    "title": r["title"],
                    "sim": r["similarity"],
                    "title_similarity": r["title_similarity"],
                    "abstract": r["abstract"],
                }
            )

        await conn.close()
        return matches
if False:
    embeding = np.random.rand(768)

    # Run the SQL commands now.
    res = await fetch(embeding)  # type: ignore
    res

In [46]:
from transformers import AutoTokenizer, AutoModel
model_type = 'jina'
if model_type == "bert":
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
elif model_type == "roberta":
    tokenizer = AutoTokenizer.from_pretrained('FacebookAI/roberta-base')
    model = AutoModel.from_pretrained('FacebookAI/roberta-base')  
elif model_type == 'jina':
    tokenizer = AutoTokenizer.from_pretrained('jinaai/jina-embeddings-v3')
    model = AutoModel.from_pretrained('jinaai/jina-embeddings-v3', trust_remote_code=True)
else:
    Exception("No model chosen")

TypeError: XLMRobertaLoRA.__init__() got an unexpected keyword argument 'task'

In [10]:
import torch
# Sample text
text = "SciBERT is a pretrained transformer model for scientific text."

# Tokenize the input text
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Forward pass to get the embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract the last hidden states (embeddings)
last_hidden_states = outputs.last_hidden_state

# Optionally, get the embeddings for the [CLS] token
cls_embedding = last_hidden_states[:, 0, :]

# Print the shape of the embeddings
print("Shape of last hidden states:", last_hidden_states.shape)
print("Shape of CLS embedding:", cls_embedding.shape)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Shape of last hidden states: torch.Size([1, 14, 768])
Shape of CLS embedding: torch.Size([1, 768])


In [11]:
import pandas as pd
import numpy as np
df = pd.read_csv("test.csv")
df["embedding"] = [np.random.rand(768) for _ in range(len(df))]
with torch.no_grad():
    embeds = model(**tokenizer(df["abstract"].to_list(), return_tensors='pt', padding=True, truncation=True)).last_hidden_state[:, 0, :]
    for idx, emb in enumerate(embeds):
        df.at[idx, "embedding"] = emb.numpy()
df

,doi,title,abstract,content,citation_text,embedding
0,1a,test10,An article on NLP discussing transformers,the long article1,citation,"[-0.1714096, 0.5485956, -0.2250522, 0.00396031..."
1,2a,test20,An article on biology discussing frogs,the long article2,citation,"[-0.6305636, -0.57029116, 0.019760396, 0.61680..."
2,3a,test30,An article on AI discussing Neural networks,the long article3,citation,"[-0.7799448, -0.13999197, 0.41758755, 0.049859..."


In [103]:
# await reset_database() #[-0.1714102, 0.548595, -0.22505158, 0.003960
# await insert_into_papers(df) #[-0.1714096, 0.5485956, -0.22505

In [159]:
async def fetch_all():
    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database.
        conn: asyncpg.Connection = await connector.connect_async(
            f"{project_id}:{region}:{instance_name}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{database_user}",
            password=f"{database_password}",
            db=f"{database_name}",
        )

        await register_vector(conn)
        # Find similar products to the query using cosine similarity search
        # over all vector embeddings. This new feature is provided by `pgvector`.
        results = await conn.fetch(
            """
                            SELECT *
                            FROM papers
                            """
        )

        if len(results) == 0:
            return []
        matches = []
        for r in results:
            # Collect the description for all the matched similar toy products.
            matches.append(
                {
                    "doi": r["doi"],
                    "title": r["title"],
                    "abstract": r["abstract"],
                    "content": r["content"],
                    "emb": r["embedding"],
                    "id": r["id"],
                }
            )

        await conn.close()
        return matches
matches = await fetch_all()
len(matches)

10000

In [106]:
df_abstract = pd.read_csv("./data/ml_100_abstracts.csv")
df_abstract["doi"] = None
df_abstract["title"] = None
df_abstract["content"] = "None"
df_abstract["citation_text"] = "None"
for i in range(len(df_abstract)):
    df_abstract.loc[i, "doi"] = "paper" + str(i)
    df_abstract.loc[i, "title"] = str(i)
    df_abstract.loc[i, "content"] = str(i)
    df_abstract.loc[i, "citation_text"] = str(i)
df_abstract.rename(columns={'Abstract': 'abstract'}, inplace=True)
df_abstract

,Unnamed: 0,abstract,doi,title,content,citation_text
0,0,The problem of statistical learning is to co...,paper0,0,0,0
1,1,"In a sensor network, in practice, the commun...",paper1,1,1,1
2,2,We analyze the generalization performance of...,paper2,2,2,2
3,3,In the process of training Support Vector Ma...,paper3,3,3,3
4,4,Probabilistic graphical models (PGMs) have b...,paper4,4,4,4
...,...,...,...,...,...,...
95,95,PRISM is an extension of Prolog with probabi...,paper95,95,95,95
96,96,"Recently, applying the novel data mining tec...",paper96,96,96,96
97,97,The typical behavior of optimal solutions to...,paper97,97,97,97
98,98,In this paper we analyze judgement aggregati...,paper98,98,98,98


In [149]:
import pandas as pd
def add_embedings(df: pd.DataFrame, model, tokenizer, field_to_embedd="abstract", embedding_field_name="embedding"):
    # df["embedding"] = None
    with torch.no_grad():
        inputs = tokenizer(df[field_to_embedd].to_list(), return_tensors='pt', padding=True, truncation=True)
        embeds = model(**inputs, task="retrieval.passage").last_hidden_state[:, 0, :]
        df.insert(len(df), embedding_field_name, embeds.tolist())
        # assert len(df) == len(embeds)
        # for idx, emb in enumerate(embeds):
        #     df.at[idx, "embedding"] = emb.numpy()
df_abstract

,Unnamed: 0,abstract,doi,title,content,citation_text
0,0,The problem of statistical learning is to co...,paper0,0,0,0
1,1,"In a sensor network, in practice, the commun...",paper1,1,1,1
2,2,We analyze the generalization performance of...,paper2,2,2,2
3,3,In the process of training Support Vector Ma...,paper3,3,3,3
4,4,Probabilistic graphical models (PGMs) have b...,paper4,4,4,4
...,...,...,...,...,...,...
95,95,PRISM is an extension of Prolog with probabi...,paper95,95,95,95
96,96,"Recently, applying the novel data mining tec...",paper96,96,96,96
97,97,The typical behavior of optimal solutions to...,paper97,97,97,97
98,98,In this paper we analyze judgement aggregati...,paper98,98,98,98


In [52]:
# reset_database() #[-0.1714102, 0.548595, -0.22505158, 0.003960
# await insert_into_papers(df_abstract) #[-0.1714096, 0.5485956, -0.22505

C:\Users\User\AppData\Local\Temp\ipykernel_9672\1845722292.py:1: RuntimeWarning: coroutine 'reset_database' was never awaited
  reset_database() #[-0.1714102, 0.548595, -0.22505158, 0.003960
c:\Users\User\GaTech\hackathon\i-cite\venv\Lib\site-packages\google\cloud\sql\connector\instance.py:330: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  expiration = x509.not_valid_after


DROP TABLE


In [142]:
text = ["summarize the papers related to linear regression",
        "quantum mechanics is a very interesting field plese get mo som epapers related to that subject",
        "What are transformers what are examples of transformer, encoder decoder models"
        ]

async def get_similar(text, nbr_articles=5):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    # Forward pass to get the embeddings
    with torch.no_grad():
        outputs = model(**inputs, task="retrieval.query")

    # Extract the last hidden states (embeddings)
    last_hidden_states = outputs.last_hidden_state

    # Optionally, get the embeddings for the [CLS] token
    cls_embedding = last_hidden_states[:, 0, :]
    return await fetch(cls_embedding[0], nbr_articles=nbr_articles, DESC=True)

for sent in text:
    print(sent)
    res = await get_similar(sent)
    for i in res:
        print(i)
    

Flash attention implementation does not support kwargs: task


summarize the papers related to linear regression


Flash attention implementation does not support kwargs: task


{'title': 'Near-optimal Coresets For Least-Squares Regression', 'sim': 0.678784746859353, 'title_similarity': None, 'abstract': '  We study (constrained) least-squares regression as well as multiple response\nleast-squares regression and ask the question of whether a subset of the data,\na coreset, suffices to compute a good approximate solution to the regression.\nWe give deterministic, low order polynomial-time algorithms to construct such\ncoresets with approximation guarantees, together with lower bounds indicating\nthat there is not much room for improvement upon our results.\n'}
{'title': 'Low rank approximation and decomposition of large matrices using error\n  correcting codes', 'sim': 0.6493639302953358, 'title_similarity': None, 'abstract': '  Low rank approximation is an important tool used in many applications of\nsignal processing and machine learning. Recently, randomized sketching\nalgorithms were proposed to effectively construct low rank approximations and\nobtain appr

Flash attention implementation does not support kwargs: task


{'title': 'Strange Bedfellows: Quantum Mechanics and Data Mining', 'sim': 0.717725049345647, 'title_similarity': None, 'abstract': '  Last year, in 2008, I gave a talk titled {\\it Quantum Calisthenics}. This\nyear I am going to tell you about how the work I described then has spun off\ninto a most unlikely direction. What I am going to talk about is how one maps\nthe problem of finding clusters in a given data set into a problem in quantum\nmechanics. I will then use the tricks I described to let quantum evolution lets\nthe clusters come together on their own.\n'}
{'title': 'Quantum machine learning: a classical perspective', 'sim': 0.7173719536168784, 'title_similarity': None, 'abstract': '  Recently, increased computational power and data availability, as well as\nalgorithmic advances, have led machine learning techniques to impressive\nresults in regression, classification, data-generation and reinforcement\nlearning tasks. Despite these successes, the proximity to the physical lim

In [58]:
df_json = pd.read_json("./data/ml_papers.json")

In [144]:
df_json_filterd = df_json[['doi', 'title', 'abstract', 'id']]
df_json_filterd["content"] = "None"
df_json_filterd["citation_text"] = "None"
df_json_filterd = df_json_filterd.drop_duplicates(subset='doi', keep='first')

C:\Users\User\AppData\Local\Temp\ipykernel_9672\2352425748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_json_filterd["content"] = "None"
C:\Users\User\AppData\Local\Temp\ipykernel_9672\2352425748.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_json_filterd["citation_text"] = "None"


In [157]:
# import warnings
# from cryptography.utils import CryptographyDeprecationWarning 
# warnings.filterwarnings("ignore", category=CryptographyDeprecationWarning)
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
dois = [item['doi'] for item in await fetch_all()]
print(dois)
chunk_size = 5
seq = df_json_filterd[~df_json_filterd['doi'].isna()][5000:10000]
seq = seq[~seq['doi'].isin(dois)]
for idx, chunk in enumerate(chunker(seq=seq, size=chunk_size)):
    print("processing ", idx*chunk_size, "to", (idx+1)*chunk_size)
    add_embedings(chunk, model=model, tokenizer=tokenizer)
    add_embedings(chunk, model=model, tokenizer=tokenizer, field_to_embedd="title", embedding_field_name="title_embedding")
    await insert_into_papers(chunk)


['10.1109/TSMC.2021.3074496', '10.1109/TWC.2016.2636139', '10.1109/TSP.2016.2628348', '10.1145/2978578', '10.1109/TSP.2017.2656847', '10.1109/TAC.2018.2813009', '10.1016/j.csda.2018.03.015', '10.1016/j.jides.2016.11.001', '10.1145/1569901.1570100', '10.4236/ijmnta.2014.34020', '10.1186/s13062-017-0203-4', '10.1016/j.cviu.2017.05.007', '10.18653/v1/W16-0533', '10.3390/e18120442', '10.1209/0295-5075/117/38002', '10.1109/ICASSP.2017.7952599', '10.1007/s10710-010-9103-4', '10.1109/LSP.2016.2589962', '10.1109/ACSSC.2017.8335713', '10.1109/ACCESS.2020.2976199', '10.1007/s00365-019-09489-8', '10.1109/TNNLS.2016.2572310', '10.13140/RG.2.1.2436.5683', '10.1142/S0129183109013613', '10.1111/j.1467-9868.2009.00698.x', '10.1143/JPSJ.77.094801', '10.1109/TIT.2010.2090235', '10.1016/j.isprsjprs.2015.01.006', '10.1109/JSTSP.2018.2846218', '10.1007/s10994-019-05856-5', '10.18653/v1/W19-8630', '10.1103/PhysRevC.80.044332', '10.1109/ISCC.2008.4625611', '10.3847/2041-8213/aa603d', '10.4204/EPTCS.215.7', '

Flash attention implementation does not support kwargs: task
Flash attention implementation does not support kwargs: task
Flash attention implementation does not support kwargs: task


processing  5 to 10


Flash attention implementation does not support kwargs: task
Flash attention implementation does not support kwargs: task


processing  10 to 15


...

Flash attention implementation does not support kwargs: task
Flash attention implementation does not support kwargs: task


processing  3045 to 3050


Flash attention implementation does not support kwargs: task
Flash attention implementation does not support kwargs: task


processing  3050 to 3055


Flash attention implementation does not support kwargs: task


In [ ]:

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))
dois = [item['doi'] for item in await fetch_all() if item['id'] == None]
print(len(dois))
chunk_size = 5
seq = df_json_filterd[~df_json_filterd['doi'].isna()][5000:10000]
seq = seq[seq['doi'].isin(dois)]
for idx, chunk in enumerate(chunker(seq=seq, size=chunk_size)):
    print("processing ", idx*chunk_size, "to", (idx+1)*chunk_size)
    add_embedings(chunk, model=model, tokenizer=tokenizer, field_to_embedd="title", embedding_field_name="title_embedding")
    await update_papers(chunk)

Flash attention implementation does not support kwargs: task


1945
processing  0 to 5


Flash attention implementation does not support kwargs: task


processing  5 to 10


Flash attention implementation does not support kwargs: task


...

processing  755 to 760


Flash attention implementation does not support kwargs: task


processing  760 to 765


Flash attention implementation does not support kwargs: task


processing  765 to 770


In [145]:
df_json_filterd


,doi,title,abstract,id,content,citation_text
0,10.1109/ITW.2007.4313111,Learning from compressed observations,The problem of statistical learning is to co...,0704.0671,None,None
1,10.1109/TSP.2008.920143,Sensor Networks with Random Links: Topology De...,"In a sensor network, in practice, the commun...",0704.0954,None,None
2,10.1143/JPSJ.76.114001,Statistical Mechanics of Nonlinear On-line Lea...,We analyze the generalization performance of...,0705.2318,None,None
3,10.1109/ICTAI.2007.99,A Novel Model of Working Set Selection for SMO...,In the process of training Support Vector Ma...,0706.0585,None,None
4,10.1371/journal.pcbi.0030252,Getting started in probabilistic graphical models,Probabilistic graphical models (PGMs) have b...,0706.2040,None,None
...,...,...,...,...,...,...
24983,10.1016/j.amc.2004.09.003,A Model for Prejudiced Learning in Noisy Envir...,Based on the heuristics that maintaining pre...,nlin/0306055,None,None
24984,10.1016/j.physd.2005.06.031,Stability and Diversity in Collective Adaptation,We derive a class of macroscopic differentia...,nlin/0408039,None,None
24985,10.1007/978-3-540-73133-7_1,Structural Inference of Hierarchies in Networks,One property of networks that has received c...,physics/0610051,None,None
24986,10.1073/pnas.0406011101,Parametric Inference for Biological Sequence A...,One of the major successes in computational ...,q-bio/0401033,None,None
